# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 15 2022 9:22AM,250785,16,8775527,Sartorius Bioprocess Solutions,Released
1,Nov 15 2022 9:22AM,250785,16,8776055,Sartorius Bioprocess Solutions,Released
2,Nov 15 2022 9:22AM,250783,16,8775396,Sartorius Bioprocess Solutions,Released
3,Nov 15 2022 9:17AM,250784,12,U1109,"Uniderm USA, Inc",Released
4,Nov 15 2022 9:17AM,250784,12,U1110,"Uniderm USA, Inc",Released
5,Nov 15 2022 9:02AM,250778,12,HH8340,Hush Hush,Released
6,Nov 15 2022 8:56AM,250780,10,0086172366,ISDIN Corporation,Executing
7,Nov 15 2022 8:56AM,250780,10,0086172363,ISDIN Corporation,Executing
8,Nov 15 2022 8:56AM,250780,10,0086172365,ISDIN Corporation,Executing
9,Nov 15 2022 8:56AM,250780,10,0086172367,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,250780,Executing,11
19,250780,Released,3
20,250783,Released,1
21,250784,Released,2
22,250785,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250779,NaN,NaN,60.0
250780,NaN,11.0,3.0
250783,NaN,NaN,1.0
250784,NaN,NaN,2.0
250785,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0.0,0.0,48.0
250674,0.0,1.0,0.0
250682,0.0,0.0,64.0
250695,0.0,0.0,1.0
250703,23.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0,0,48
250674,0,1,0
250682,0,0,64
250695,0,0,1
250703,23,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,0,0,48
1,250674,0,1,0
2,250682,0,0,64
3,250695,0,0,1
4,250703,23,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,,,48
1,250674,,1,
2,250682,,,64
3,250695,,,1
4,250703,23,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions
2,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions
3,Nov 15 2022 9:17AM,250784,12,"Uniderm USA, Inc"
5,Nov 15 2022 9:02AM,250778,12,Hush Hush
6,Nov 15 2022 8:56AM,250780,10,ISDIN Corporation
20,Nov 15 2022 8:56AM,250779,10,ISDIN Corporation
80,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation
142,Nov 15 2022 7:37AM,250771,20,"Exact-Rx, Inc."
151,Nov 15 2022 4:39AM,250769,10,ISDIN Corporation
166,Nov 14 2022 6:13PM,250768,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions,,,2
1,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions,,,1
2,Nov 15 2022 9:17AM,250784,12,"Uniderm USA, Inc",,,2
3,Nov 15 2022 9:02AM,250778,12,Hush Hush,,,1
4,Nov 15 2022 8:56AM,250780,10,ISDIN Corporation,,11,3
5,Nov 15 2022 8:56AM,250779,10,ISDIN Corporation,,,60
6,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation,,,62
7,Nov 15 2022 7:37AM,250771,20,"Exact-Rx, Inc.",,4,5
8,Nov 15 2022 4:39AM,250769,10,ISDIN Corporation,,1,14
9,Nov 14 2022 6:13PM,250768,15,"Brookfield Pharmaceuticals, LLC",,,59


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions,2,,
1,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions,1,,
2,Nov 15 2022 9:17AM,250784,12,"Uniderm USA, Inc",2,,
3,Nov 15 2022 9:02AM,250778,12,Hush Hush,1,,
4,Nov 15 2022 8:56AM,250780,10,ISDIN Corporation,3,11,
5,Nov 15 2022 8:56AM,250779,10,ISDIN Corporation,60,,
6,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation,62,,
7,Nov 15 2022 7:37AM,250771,20,"Exact-Rx, Inc.",5,4,
8,Nov 15 2022 4:39AM,250769,10,ISDIN Corporation,14,1,
9,Nov 14 2022 6:13PM,250768,15,"Brookfield Pharmaceuticals, LLC",59,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions,2,,
1,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions,1,,
2,Nov 15 2022 9:17AM,250784,12,"Uniderm USA, Inc",2,,
3,Nov 15 2022 9:02AM,250778,12,Hush Hush,1,,
4,Nov 15 2022 8:56AM,250780,10,ISDIN Corporation,3,11,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN
1,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Nov 15 2022 9:17AM,250784,12,"Uniderm USA, Inc",2.0,NaN,NaN
3,Nov 15 2022 9:02AM,250778,12,Hush Hush,1.0,NaN,NaN
4,Nov 15 2022 8:56AM,250780,10,ISDIN Corporation,3.0,11.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0
1,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Nov 15 2022 9:17AM,250784,12,"Uniderm USA, Inc",2.0,0.0,0.0
3,Nov 15 2022 9:02AM,250778,12,Hush Hush,1.0,0.0,0.0
4,Nov 15 2022 8:56AM,250780,10,ISDIN Corporation,3.0,11.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1755295,237.0,12.0,0.0
12,501562,3.0,0.0,0.0
15,501438,107.0,0.0,0.0
16,752334,4.0,0.0,0.0
19,250703,0.0,1.0,23.0
20,501466,6.0,4.0,0.0
21,501431,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1755295,237.0,12.0,0.0
1,12,501562,3.0,0.0,0.0
2,15,501438,107.0,0.0,0.0
3,16,752334,4.0,0.0,0.0
4,19,250703,0.0,1.0,23.0
5,20,501466,6.0,4.0,0.0
6,21,501431,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,237.0,12.0,0.0
1,12,3.0,0.0,0.0
2,15,107.0,0.0,0.0
3,16,4.0,0.0,0.0
4,19,0.0,1.0,23.0
5,20,6.0,4.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,237.0
1,12,Released,3.0
2,15,Released,107.0
3,16,Released,4.0
4,19,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,23.0,0.0,0.0
Executing,12.0,0.0,0.0,0.0,1.0,4.0,1.0
Released,237.0,3.0,107.0,4.0,0.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,12.0,0.0,0.0,0.0,1.0,4.0,1.0
2,Released,237.0,3.0,107.0,4.0,0.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,12.0,0.0,0.0,0.0,1.0,4.0,1.0
2,Released,237.0,3.0,107.0,4.0,0.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()